In [ ]:
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.datasets import imdb
from keras.utils import pad_sequences

In [ ]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        # embed_dim: This parameter specifies the dimensionality of the input and output embeddings.
        # num_heads: This parameter controls the number of attention heads in the multi-head attention layer.
        # ff_dim: This parameter specifies the dimensionality of the feedforward network.
        # rate: This parameter controls the dropout rate, which is used to regularize the network and prevent overfitting.

        super().__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # This creates a MultiHeadAttention layer, responsible for learning long-range dependencies within sequences.
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        # self.ffn: This creates a feedforward network, often used for additional nonlinear transformations.
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        # self.layernorm1 and self.layernorm2: These create LayerNormalization layers, which help stabilize training and improve convergence.
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        # self.dropout1 and self.dropout2: These create Dropout layers, randomly setting a portion of inputs to zero during training to prevent overfitting.

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        # Applies multi-head attention to the input sequence, allowing different parts of the sequence to interact with each other.
        attn_output = self.dropout1(attn_output, training=training)
        # Applies dropout to the attention output.
        out1 = self.layernorm1(inputs + attn_output)
        # Adds the attention output to the original input and applies layer normalization. This residual connection helps prevent vanishing gradients and improves training stability.
        ffn_output = self.ffn(out1)
        # Passes the normalized output through the feedforward network.
        ffn_output = self.dropout2(ffn_output, training=training)
        # Applies dropout to the feedforward output.
        return self.layernorm2(out1 + ffn_output)
        # Adds the feedforward output to the previous layer's output and applies final layer normalization before returning the result.

In [ ]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        # maxlen: The maximum length of the input sequences the model will handle.
        # vocab_size: The total number of unique tokens (words) in the vocabulary.
        # embed_dim: The dimensionality of the embeddings (how each token and its position will be represented as a vector).
        super().__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        # An Embedding layer that maps each token in the input sequence
        # to a dense vector of size embed_dim.
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)
        # An Embedding layer that maps each position in the sequence
        # (from 0 to maxlen-1) to a dense vector of size embed_dim.

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        # Extracts the actual length of the current input sequence.
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # Creates a tensor of positions from 0 to maxlen-1.
        positions = self.pos_emb(positions)
        # Looks up the position embeddings for each position in the sequence.
        x = self.token_emb(x)
        # Looks up the token embeddings for each token in the input sequence.
        return x + positions
        # Adds the token embeddings and position embeddings element-wise,
        # resulting in a combined representation that captures both word
        # meaning and positional information.

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = pad_sequences(x_train, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

17464789/17464789 [==============================] - 0s 0us/step
25000 Training sequences
25000 Validation sequences


In [ ]:
x_train.shape

(25000, 200)

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
history = model.fit(
    x_train, y_train, batch_size=32,
    epochs=10,
    validation_data=(x_val, y_val)
)

Epoch 1/10
782/782 [==============================] - 136s 169ms/step - loss: 0.3919 - accuracy: 0.8105 - val_loss: 0.2901 - val_accuracy: 0.8776
Epoch 2/10
782/782 [==============================] - 120s 153ms/step - loss: 0.1991 - accuracy: 0.9232 - val_loss: 0.3143 - val_accuracy: 0.8732
Epoch 3/10
782/782 [==============================] - 128s 164ms/step - loss: 0.1304 - accuracy: 0.9537 - val_loss: 0.3935 - val_accuracy: 0.8636
Epoch 4/10
782/782 [==============================] - 129s 165ms/step - loss: 0.0834 - accuracy: 0.9721 - val_loss: 0.5215 - val_accuracy: 0.8535
Epoch 5/10
782/782 [==============================] - 120s 153ms/step - loss: 0.0562 - accuracy: 0.9826 - val_loss: 0.6220 - val_accuracy: 0.8472
Epoch 6/10
782/782 [==============================] - 130s 166ms/step - loss: 0.0364 - accuracy: 0.9896 - val_loss: 0.6928 - val_accuracy: 0.8443
Epoch 7/10
782/782 [==============================] - 129s 165ms/step - loss: 0.0338 - accuracy: 0.9901 - val_loss: 0.7609 -

In [ ]:
x_train

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)